### Utilize advanced data analysis to identify top performers and team performance over the past three years in IPL.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
#importing the Datasets
match_sum = pd.read_csv(r'C:\CS\Data science\codebasics\C_10_ipl\datasets\dim_match_summary.csv')
players = pd.read_csv(r'C:\CS\Data science\codebasics\C_10_ipl\datasets\dim_players.csv')
bat_sum = pd.read_csv(r'C:\CS\Data science\codebasics\C_10_ipl\datasets\fact_bating_summary.csv')
bowl_sum = pd.read_csv(r'C:\CS\Data science\codebasics\C_10_ipl\datasets\fact_bowling_summary.csv')

### Understanding the Dataset

In [3]:
match_sum.shape,players.shape,bat_sum.shape,bowl_sum.shape

((206, 6), (292, 5), (3268, 11), (2436, 14))

In [4]:
match_sum.duplicated().sum(),players.duplicated().sum(),bat_sum.duplicated().sum(),bowl_sum.duplicated().sum()

(0, 0, 0, 0)

In [5]:
match_sum.isnull().sum().sum(),players.isnull().sum().sum(),bat_sum.isnull().sum().sum(),\
bowl_sum.isnull().sum().sum()

(0, 0, 0, 0)

In [6]:
match_sum.head(2)


,team1,team2,winner,margin,matchDate,match_id
0,Super Kings,KKR,Super Kings,27 runs,"Oct 15, 2021",T203817
1,Capitals,KKR,KKR,3 wickets,"Oct 13, 2021",T206442


In [7]:
match_sum.dtypes

team1        object
team2        object
winner       object
margin       object
matchDate    object
match_id     object
dtype: object

In [9]:
match_sum.loc[match_sum['match_id']=='T208201']
#Need to edit the date to May 28 or else it will cause error in further analysis since the match
# date should be single date but in this case t was provided as multiple dates.

,team1,team2,winner,margin,matchDate,match_id
133,Titans,Super Kings,Super Kings,5 wickets,"May 28-29, 2023",T208201


In [10]:
match_sum.loc[match_sum['match_id']=='T208201',['matchDate']] = 'May 28, 2023'

In [11]:
match_sum.head(2)

,team1,team2,winner,margin,matchDate,match_id
0,Super Kings,KKR,Super Kings,27 runs,"Oct 15, 2021",T203817
1,Capitals,KKR,KKR,3 wickets,"Oct 13, 2021",T206442


In [12]:
match_sum.loc[match_sum['match_id']=='T208201']

,team1,team2,winner,margin,matchDate,match_id
133,Titans,Super Kings,Super Kings,5 wickets,"May 28, 2023",T208201


In [13]:
# need to convert matchDate from object to datetime

In [14]:
match_sum['matchDate'] = pd.to_datetime(match_sum['matchDate'])

In [15]:
match_sum['matchDate']

0     2021-10-15
1     2021-10-13
2     2021-10-11
3     2021-10-10
4     2021-10-08
         ...    
201   2023-04-02
202   2023-04-02
203   2023-04-01
204   2023-04-01
205   2023-03-31
Name: matchDate, Length: 206, dtype: datetime64[ns]

In [16]:
match_sum.loc[match_sum['match_id']=='T208201']

,team1,team2,winner,margin,matchDate,match_id
133,Titans,Super Kings,Super Kings,5 wickets,2023-05-28,T208201


In [17]:
match_sum.dtypes

team1                object
team2                object
winner               object
margin               object
matchDate    datetime64[ns]
match_id             object
dtype: object

### Feature Creation

#### 1.Creating a new field Season based on year where 2021 - 1(season 1), 2022 - 2 (season 2),2023 - 3(season 3). This field is used for finding the players who batted minimum 60 balls per Season and also for finding the bowlers who bowled minimum 60 balls per season. For that 1st year is extracted and then giving values 1,2 and 3.
#### 2. Creating a column for out and not_out, If out we will give value as 1 and if not out we will give value as 0. This Field is used for Finding the average, since average is the total runs scored divided by total number of innings in which the batsman got out. If we give integer 1 for out , then by taking the sum we wil get the number of innings in which that player got out.
#### 3.Here we need to calculate the total ball bowled. For that we need to multiple the integer part by six and add the decimal part to it after multiplying the decimal part by 10.



In [18]:
#1. Season
match_sum['year'] = match_sum['matchDate'].dt.year

In [19]:
match_sum.head(2)

,team1,team2,winner,margin,matchDate,match_id,year
0,Super Kings,KKR,Super Kings,27 runs,2021-10-15,T203817,2021
1,Capitals,KKR,KKR,3 wickets,2021-10-13,T206442,2021


In [20]:
match_sum.dtypes

team1                object
team2                object
winner               object
margin               object
matchDate    datetime64[ns]
match_id             object
year                  int64
dtype: object

In [21]:
conditions = [
                (match_sum['year'] == 2021),
                (match_sum['year'] == 2022),
                (match_sum['year'] == 2023)
            ]

choices = [1, 2, 3]

match_sum['season'] = np.select(conditions, choices, default=np.nan)

In [22]:
match_sum.head(2)

,team1,team2,winner,margin,matchDate,match_id,year,season
0,Super Kings,KKR,Super Kings,27 runs,2021-10-15,T203817,2021,1.0
1,Capitals,KKR,KKR,3 wickets,2021-10-13,T206442,2021,1.0


In [23]:
## Converting the datatype from int to float
match_sum['season'] = match_sum['season'].astype(int)

In [24]:
match_sum.dtypes

team1                object
team2                object
winner               object
margin               object
matchDate    datetime64[ns]
match_id             object
year                  int64
season                int32
dtype: object

In [25]:
bat_sum['out/not_out'].unique()

array(['out', 'not_out'], dtype=object)

In [26]:
bat_sum.head(2)

,match_id,match,teamInnings,battingPos,batsmanName,out/not_out,runs,balls,4s,6s,SR
0,T203817,Super Kings Vs KKR,Super Kings,1,RuturajGaikwad,out,32,27,3,1,118.51
1,T203817,Super Kings Vs KKR,Super Kings,2,FafduPlessis,out,86,59,7,3,145.76


In [27]:
condition_list =[
            bat_sum['out/not_out'] == 'out',
           # bat_sum['out/not_out'] == 'not_out'
        ]
choice_list = [1]
bat_sum['out_flag'] = np.select(condition_list,choice_list,0)

In [28]:
bat_sum.sample(2)

,match_id,match,teamInnings,battingPos,batsmanName,out/not_out,runs,balls,4s,6s,SR,out_flag
1614,T204535,RCB Vs Super Giants,Super Giants,5,DeepakHooda,out,13,14,1,0,92.85,1
2559,T203494,RCB Vs Super Giants,RCB,7,MahipalLomror,out,3,4,0,0,75,1


In [29]:
bowl_sum.head(2)

,match_id,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,T203817,Super Kings Vs KKR,KKR,ShakibAlHasan,3.0,0,33,0,11.0,6,2,3,0,0
1,T203817,Super Kings Vs KKR,KKR,ShivamMavi,4.0,0,32,1,8.0,8,1,2,0,0


### Feature 3

In [84]:
bowl_sum['balls_bowled'] = ((bowl_sum['overs'] // 1) * 6) + (round(bowl_sum['overs'] % 1,1) * 10)
bowl_sum['balls_bowled'] = bowl_sum['balls_bowled'].astype(int)
#converting the output to integer

In [85]:
bowl_sum[['overs','balls_bowled']].sample(10)

,overs,balls_bowled
111,2.0,12
2432,3.2,20
1037,3.0,18
530,3.0,18
179,4.0,24
1691,1.0,6
1657,1.2,8
1436,1.0,6
1207,4.0,24
1701,4.0,24


### Primary Insights

### Top 10 batsmen based on past 3 years total runs scored.

In [86]:
tot_runs_per_player = bat_sum.groupby(by=['batsmanName'])[['runs']].sum()
tot_runs_per_player.sort_values(by=['runs'],ascending=False).head(10)

,runs
batsmanName,
ShubmanGill,1851
FafduPlessis,1831
RuturajGaikwad,1593
KLRahul,1516
JosButtler,1509
ShikharDhawan,1392
ViratKohli,1385
SanjuSamson,1304
SuryakumarYadav,1225


#### Merging 'match_sum' and 'bat_sum' datset

In [87]:
match_bat = pd.merge(left=match_sum,right=bat_sum,on=['match_id'],how='left')
match_bat.head(2)

,team1,team2,winner,margin,matchDate,match_id,year,season,match,teamInnings,battingPos,batsmanName,out/not_out,runs,balls,4s,6s,SR,out_flag
0,Super Kings,KKR,Super Kings,27 runs,2021-10-15,T203817,2021,1,Super Kings Vs KKR,Super Kings,1,RuturajGaikwad,out,32,27,3,1,118.51,1
1,Super Kings,KKR,Super Kings,27 runs,2021-10-15,T203817,2021,1,Super Kings Vs KKR,Super Kings,2,FafduPlessis,out,86,59,7,3,145.76,1


#### Creating 3 dataset based for each season

In [88]:
match_bat_s1 = match_bat.loc[match_bat['season'] == 1]
match_bat_s2 = match_bat.loc[match_bat['season'] == 2]
match_bat_s3 = match_bat.loc[match_bat['season'] == 3]

In [89]:
match_bat.shape,match_bat_s1.shape,match_bat_s2.shape,match_bat_s3.shape

((3268, 19), (919, 19), (1176, 19), (1173, 19))

In [90]:
match_bat_s1.columns

Index(['team1', 'team2', 'winner', 'margin', 'matchDate', 'match_id', 'year',
       'season', 'match', 'teamInnings', 'battingPos', 'batsmanName',
       'out/not_out', 'runs', 'balls', '4s', '6s', 'SR', 'out_flag'],
      dtype='object')

In [91]:
def innings(df):
    dic = {'batsmanName':'count','runs':'sum','balls':'sum','out_flag':'sum'}
    match_bat = df.groupby(by=['batsmanName']).agg(dic)
    match_bat = match_bat.rename(columns = {'batsmanName':'innings'})
    match_bat = match_bat.loc[match_bat['balls']>=60]
    match_bat.reset_index(inplace=True)
    print(match_bat['balls'].min()) # checking the min value is it minimum 60 or not
    return(match_bat)


In [92]:
match_bat_s1_60 = innings(match_bat_s1)
match_bat_s2_60 = innings(match_bat_s2)
match_bat_s3_60 = innings(match_bat_s3)

62
61
60


In [93]:
match_bat_s1_60.head(2)

,batsmanName,innings,runs,balls,out_flag
0,ABdeVilliers,14,313,211,10
1,AbdulSamad,10,111,87,9


In [94]:
match_bat_s2_60.head(2)

,batsmanName,innings,runs,balls,out_flag
0,AaronFinch,5,86,61,5
1,AbhinavManohar,7,108,75,6


In [95]:
match_bat_s3_60.head(2)

,batsmanName,innings,runs,balls,out_flag
0,AbdulSamad,8,169,128,4
1,AbhinavManohar,6,114,83,6


#### Now concating all the above tables an group by batsmanName. Then taking the tat runs scored and balls taken.

In [96]:
match_bat_60 = pd.concat([match_bat_s1_60,match_bat_s2_60,match_bat_s3_60])
dic= {'runs':'sum','balls':'sum','innings':'sum','out_flag':'sum'}
match_bat_60 = match_bat_60.groupby(by=['batsmanName']).agg(dic)
match_bat_60

,runs,balls,innings,out_flag
batsmanName,,,,
ABdeVilliers,313,211,14,10
AaronFinch,86,61,5,5
AbdulSamad,280,215,18,13
AbhinavManohar,222,158,13,12
AbhishekSharma,745,546,31,30
...,...,...,...,...
VijayShankar,301,188,10,8
ViratKohli,1385,1090,45,41
WashingtonSundar,161,129,12,11


### Top 10 batsmen based on past 3 years 1. batting average 2. Strike rate. (min 60 balls faced in each season)

##### Calculating the batting average = Total runs / Total times he got out

In [97]:
# Calculating the batting average = Total runs / Total times he got out
match_bat_60['bat_avg'] = round ( ( match_bat_60['runs'] / match_bat_60['out_flag'] ) ,2 )
# Calculating the strike rate = Total runs / Total balls faced
match_bat_60['strike_rate'] = round ( ( match_bat_60['runs'] / match_bat_60['balls'] * 100) , 2 )
match_bat_60

,runs,balls,innings,out_flag,bat_avg,strike_rate
batsmanName,,,,,,
ABdeVilliers,313,211,14,10,31.30,148.34
AaronFinch,86,61,5,5,17.20,140.98
AbdulSamad,280,215,18,13,21.54,130.23
AbhinavManohar,222,158,13,12,18.50,140.51
AbhishekSharma,745,546,31,30,24.83,136.45
...,...,...,...,...,...,...
VijayShankar,301,188,10,8,37.62,160.11
ViratKohli,1385,1090,45,41,33.78,127.06
WashingtonSundar,161,129,12,11,14.64,124.81


In [98]:
print('Top 10 batsman based on the batting average')
match_bat_60.sort_values(by=['bat_avg'],ascending=False).head(10)

Top 10 batsman based on the batting average


,runs,balls,innings,out_flag,bat_avg,strike_rate
batsmanName,,,,,,
SaurabhTiwary,115,97,4,2,57.50,118.56
KLRahul,1516,1148,37,30,50.53,132.06
CameronGreen,452,282,16,9,50.22,160.28
RinkuSingh,648,434,21,13,49.85,149.31
HeinrichKlaasen,448,253,11,9,49.78,177.08
DevonConway,924,654,22,19,48.63,141.28
SaiSudharsan,507,370,13,11,46.09,137.03
FafduPlessis,1831,1300,46,42,43.60,140.85
DavidMiller,864,628,37,20,43.20,137.58


In [99]:
print('Top 10 batsman based on the strike_rate')
match_bat_60.sort_values(by=['strike_rate'],ascending=False).head(10)

Top 10 batsman based on the strike_rate


,runs,balls,innings,out_flag,bat_avg,strike_rate
batsmanName,,,,,,
TimDavid,417,232,23,14,29.79,179.74
HeinrichKlaasen,448,253,11,9,49.78,177.08
LiamLivingstone,716,411,23,20,35.80,174.21
DhruvJurel,152,88,11,7,21.71,172.73
RashidKhan,213,128,18,11,19.36,166.41
PhilSalt,218,133,9,8,27.25,163.91
EvinLewis,151,93,5,5,30.20,162.37
GlennMaxwell,1214,752,41,35,34.69,161.44
SuryakumarYadav,1225,763,38,35,35.00,160.55


In [100]:
#### Merging 'match_sum' and 'bowl_sum' datset

In [101]:
match_bowl = pd.merge(left=match_sum,right=bowl_sum,on=['match_id'],how='left')
match_bowl.head(2)

,team1,team2,winner,margin,matchDate,match_id,year,season,match,bowlingTeam,...,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,balls_bowled
0,Super Kings,KKR,Super Kings,27 runs,2021-10-15,T203817,2021,1,Super Kings Vs KKR,KKR,...,0,33,0,11.0,6,2,3,0,0,18
1,Super Kings,KKR,Super Kings,27 runs,2021-10-15,T203817,2021,1,Super Kings Vs KKR,KKR,...,0,32,1,8.0,8,1,2,0,0,24


In [102]:
match_bowl.columns

Index(['team1', 'team2', 'winner', 'margin', 'matchDate', 'match_id', 'year',
       'season', 'match', 'bowlingTeam', 'bowlerName', 'overs', 'maiden',
       'runs', 'wickets', 'economy', '0s', '4s', '6s', 'wides', 'noBalls',
       'balls_bowled'],
      dtype='object')

In [103]:
match_bowl_s1 = match_bowl.loc[match_bowl['season'] == 1]
match_bowl_s2 = match_bowl.loc[match_bowl['season'] == 2]
match_bowl_s3 = match_bowl.loc[match_bowl['season'] == 3]

In [104]:
match_bowl.shape,match_bowl_s1.shape,match_bowl_s2.shape,match_bowl_s3.shape

((2436, 22), (687, 22), (857, 22), (892, 22))

In [105]:
def bowl(df):
    dic = {'bowlerName':'count','wickets':'sum','balls_bowled':'sum','runs':'sum'}
    match_bowl = df.groupby(by=['bowlerName']).agg(dic)
    match_bowl = match_bowl.rename(columns = {'bowlerName':'innings'})
    match_bowl = match_bowl.loc[match_bowl['balls_bowled']>=60]
    match_bowl.reset_index(inplace=True)
    print(match_bowl['balls_bowled'].min()) # checking the min value is it minimum 60 or not
    return(match_bowl)


In [106]:
match_bowl_s1.columns

Index(['team1', 'team2', 'winner', 'margin', 'matchDate', 'match_id', 'year',
       'season', 'match', 'bowlingTeam', 'bowlerName', 'overs', 'maiden',
       'runs', 'wickets', 'economy', '0s', '4s', '6s', 'wides', 'noBalls',
       'balls_bowled'],
      dtype='object')

In [107]:
match_bowl_s1_60 = bowl(match_bowl_s1)
match_bowl_s2_60 = bowl(match_bowl_s2)
match_bowl_s3_60 = bowl(match_bowl_s3)

60
66
60


In [108]:
match_bowl_s1_60.sample(2)

,bowlerName,innings,wickets,balls_bowled,runs
55,SamCurran,9,9,198,328
38,MoeenAli,10,6,152,161


#### Now concating all the above tables and group by bowlerName. Then taking the  runs scored and balls taken.

In [109]:
match_bowl_60 = pd.concat([match_bowl_s1_60,match_bowl_s2_60,match_bowl_s3_60])
dic= {'wickets':'sum','balls_bowled':'sum','runs':'sum'}
match_bowl_60 = match_bowl_60.groupby(by=['bowlerName']).agg(dic)
match_bowl_60.head(2)

,wickets,balls_bowled,runs
bowlerName,,,
AbhishekSharma,2,72,124
AdamMilne,3,84,131


#### Top 10 bowlers based on past 3 years 1. by bowling average and 2. by economy rate.(min 60 balls bowled in each season.)

##### bowling average = runs conceded divided by total wickets taken
##### economy rate  = total runs conceede divided by overs
##### where , overs = balls bowled / 6

In [110]:
match_bowl_60['bowl_avg'] = round ( ( match_bowl_60['runs'] / match_bowl_60['wickets'] ) , 2 )
match_bowl_60['econ_rate'] = round ( ( match_bowl_60['runs'] / match_bowl_60['balls_bowled'] * 6 ) , 2 )

In [111]:
print('Top 10 bowlers based on past 3 years by bowling average are')
match_bowl_60.sort_values(by=['bowl_avg'],ascending=True).head(10)

Top 10 bowlers based on past 3 years by bowling average are


,wickets,balls_bowled,runs,bowl_avg,econ_rate
bowlerName,,,,,
MoisesHenriques,4,60,45,11.25,4.50
MarkWood,11,96,130,11.82,8.12
MohitSharma,27,265,361,13.37,8.17
AkashMadhwal,14,153,219,15.64,8.59
MichaelBracewell,6,66,95,15.83,8.64
ChrisWoakes,5,66,82,16.40,7.45
MitchellMarsh,16,193,272,17.00,8.46
MohsinKhan,17,258,304,17.88,7.07
NathanCoulter-Nile,7,120,127,18.14,6.35


In [112]:
print('Top 10 bowlers based on past 3 years by economy are')
match_bowl_60.sort_values(by=['econ_rate'],ascending=True).head(10)

Top 10 bowlers based on past 3 years by economy are


,wickets,balls_bowled,runs,bowl_avg,econ_rate
bowlerName,,,,,
MoisesHenriques,4,60,45,11.25,4.50
NathanCoulter-Nile,7,120,127,18.14,6.35
SunilNarine,36,960,1056,29.33,6.60
DavidWilley,4,156,177,44.25,6.81
MitchellSantner,7,186,211,30.14,6.81
MoeenAli,21,427,501,23.86,7.04
MohsinKhan,17,258,304,17.88,7.07
AxarPatel,30,792,939,31.30,7.11
ShakibAlHasan,4,156,187,46.75,7.19


### Top 10 bowlers based on past 3 years total wickets taken.

In [55]:
dic= {'wickets':np.sum}
tot_wkts_per_player = bowl_sum.groupby(by=['bowlerName']).agg(dic)
tot_wkts_per_player.sort_values(by=['wickets'],ascending=False).head(10)

,wickets
bowlerName,
MohammedShami,67
YuzvendraChahal,66
HarshalPatel,65
RashidKhan,63
AveshKhan,47
KagisoRabada,45
ArshdeepSingh,45
VarunChakravarthy,44
ShardulThakur,43


### Top 5 batsmen based on past 3 years boundary % (fours and sixes).

##### Group at batsmanName level and find out the total runs, total 4s and total 6s scored by each
##### batsman

In [56]:
#dic = {'runs':np.sum,'4s':np.sum,'6s':np.sum}
bat_boundary =  bat_sum.groupby(by=['batsmanName'])[['runs','4s','6s']].sum()
bat_boundary.head(2)

,runs,4s,6s
batsmanName,,,
ABdeVilliers,313,23,16
AaronFinch,86,10,3


In [57]:
bat_boundary['runs_by_4s&6s'] = (bat_boundary['4s'] * 4) + (bat_boundary['6s'] * 6)

In [58]:
bat_boundary.head(2)

,runs,4s,6s,runs_by_4s&6s
batsmanName,,,,
ABdeVilliers,313,23,16,188
AaronFinch,86,10,3,58


In [59]:
bat_boundary['boundary %'] = round((bat_boundary['runs_by_4s&6s']/bat_boundary['runs']) * 100,2)
bat_boundary.head(2)

,runs,4s,6s,runs_by_4s&6s,boundary %
batsmanName,,,,,
ABdeVilliers,313,23,16,188,60.06
AaronFinch,86,10,3,58,67.44


In [60]:
max_runs = bat_boundary['runs'].max()
max_runs

1851

In [61]:
print(f'Since the max of Total runs scored by a player in all the 3 seasons was {max_runs},\
here we are cosidering min 10% of the max runs scored for the top batsmen with boundary %')
min_runs = round(bat_boundary['runs'].max()*10/100,0)
print(f'min runs considered for the top batsmen with boundary % is {min_runs}')

Since the max of Total runs scored by a player in all the 3 seasons was 1851,here we are cosidering min 10% of the max runs scored for the top batsmen with boundary %
min runs considered for the top batsmen with boundary % is 185.0


In [62]:
print("Top 5 batsmen based on past 3 years boundary % (fours and sixes) are as below \
with min 100 runs scored in all the 3 seasons combined")
bat_boundary_filtered = bat_boundary.loc[bat_boundary['runs']>=min_runs]
bat_boundary_filtered.sort_values(by=['boundary %'],ascending=False).head(5)

Top 5 batsmen based on past 3 years boundary % (fours and sixes) are as below with min 100 runs scored in all the 3 seasons combined


,runs,4s,6s,runs_by_4s&6s,boundary %
batsmanName,,,,,
AndreRussell,745,45,64,564,75.70
KyleMayers,365,36,22,276,75.62
YashasviJaiswal,1132,142,46,844,74.56
RashidKhan,304,20,24,224,73.68
EvinLewis,224,26,10,164,73.21


## Top 5 bowlers based on past 3 years dot ball %.

#### Here we need to calculate the total ball bowled. 
#### For that we need to multiple the integer part by six and add the decimal part to it after multiplying the decimal part by 10.

,overs,balls_bowled
887,1.0,6
1093,4.0,24
1545,4.0,24
1577,1.0,6
837,2.0,12
42,3.4,22
1414,4.0,24
1586,4.0,24
425,4.0,24
1835,3.0,18


#### Group by bowlerName and take the sum of balls_bowled and 0s

In [65]:
bowl_dot = bowl_sum.groupby(by=['bowlerName'])[['balls_bowled','0s']].sum()
bowl_dot

,balls_bowled,0s
bowlerName,,
AbdulSamad,12,2
AbhishekSharma,150,50
AdamMilne,99,40
AdamZampa,132,40
AdilRashid,60,20
...,...,...
YashDayal,276,100
YashThakur,191,70
YashasviJaiswal,1,0


####

In [66]:
max_balls = bowl_dot['balls_bowled'].max()
max_balls

1097

In [67]:
print(f'Since the max of Total balls bowled by a player in all the 3 seasons was {max_balls},\
here we are cosidering min 5% of the max runs balls bowled for the bowlers based\
on past 3 years dot ball %')
min_balls = int(round(bowl_dot['balls_bowled'].max()*10/100,0))
print(f'min balls bowled considered for the top bowler with dot ball % is {min_balls}')

Since the max of Total balls bowled by a player in all the 3 seasons was 1097,here we are cosidering min 5% of the max runs balls bowled for the bowlers basedon past 3 years dot ball %
min balls bowled considered for the top bowler with dot ball % is 110


In [68]:
bowl_dot['dot_ball_%'] = round((bowl_dot['0s']/bowl_dot['balls_bowled']) * 100,2)
bowl_dot.head(2)

,balls_bowled,0s,dot_ball_%
bowlerName,,,
AbdulSamad,12,2,16.67
AbhishekSharma,150,50,33.33


In [69]:
print(f"Top 5 Bowlers based on past 3 years dot ball % are as below \
with min {min_balls} balls bowled in all the 3 seasons combined")
bowl_dot_filtered = bowl_dot.loc[bowl_dot['balls_bowled']>=min_balls]
bowl_dot_filtered.sort_values(by=['dot_ball_%'],ascending=False).head(5)

Top 5 Bowlers based on past 3 years dot ball % are as below with min 110 balls bowled in all the 3 seasons combined


,balls_bowled,0s,dot_ball_%
bowlerName,,,
MohsinKhan,258,130,50.39
PrasidhKrishna,630,302,47.94
MohammedSiraj,918,438,47.71
MohammedShami,1072,510,47.57
HarshitRana,132,62,46.97


### Top 4 teams based on past 3 years winning %.

In [70]:
df1 = match_sum.groupby(by=['team1'])[['team1']].count()
df1.columns = ['bat_first_count']
df1 = df1.reset_index()
df1 = df1.rename(columns={'team1':'team'})
df1

,team,bat_first_count
0,Capitals,16
1,KKR,18
2,Mumbai,19
3,Punjab Kings,27
4,RCB,26
5,Royals,27
6,Sunrisers,15
7,Super Giants,14
8,Super Kings,29
9,Titans,15


In [71]:
df2 = match_sum.groupby(by=['team2'])[['team2']].count()
df2.columns = ['bat_second_count']
df2 = df2.reset_index()
df2

,team2,bat_second_count
0,Capitals,27
1,KKR,27
2,Mumbai,25
3,Punjab Kings,15
4,RCB,19
5,Royals,18
6,Sunrisers,26
7,Super Giants,15
8,Super Kings,16
9,Titans,18


In [72]:
df3 = match_sum.groupby(by=['winner'])[['winner']].count()
df3.columns = ['winner_count']
df3 = df3.reset_index()
df3

,winner,winner_count
0,Capitals,21
1,KKR,21
2,Mumbai,20
3,Punjab Kings,19
4,RCB,25
5,Royals,22
6,Sunrisers,13
7,Super Giants,17
8,Super Kings,25
9,Titans,23


#### Combining all the 3 above tabes using concat

In [73]:
teams_per = pd.concat([df1,df2['bat_second_count'],df3['winner_count']],axis=1)
teams_per

,team,bat_first_count,bat_second_count,winner_count
0,Capitals,16,27,21
1,KKR,18,27,21
2,Mumbai,19,25,20
3,Punjab Kings,27,15,19
4,RCB,26,19,25
5,Royals,27,18,22
6,Sunrisers,15,26,13
7,Super Giants,14,15,17
8,Super Kings,29,16,25
9,Titans,15,18,23


#### Total games played is the sum of count_1 and count_2

In [74]:
teams_per['Tot_games_played'] = teams_per['bat_first_count'] + teams_per['bat_second_count']
teams_per

,team,bat_first_count,bat_second_count,winner_count,Tot_games_played
0,Capitals,16,27,21,43
1,KKR,18,27,21,45
2,Mumbai,19,25,20,44
3,Punjab Kings,27,15,19,42
4,RCB,26,19,25,45
5,Royals,27,18,22,45
6,Sunrisers,15,26,13,41
7,Super Giants,14,15,17,29
8,Super Kings,29,16,25,45
9,Titans,15,18,23,33


### Now calculating the winning percentage

In [75]:
teams_per['winning_per'] = round(teams_per['winner_count']  /  teams_per['Tot_games_played'] * 100,2)
teams_per.sort_values(by = ['winning_per'],ascending=False).head(4).reset_index(drop= True)

,team,bat_first_count,bat_second_count,winner_count,Tot_games_played,winning_per
0,Titans,15,18,23,33,69.70
1,Super Giants,14,15,17,29,58.62
2,RCB,26,19,25,45,55.56
3,Super Kings,29,16,25,45,55.56


### Top 2 teams with the highest number of wins achieved by chasing targets over the past 3 years.

In [76]:
# Filtering based on the condition data in team2 matches with corresponding data in winner
bat_chase = match_sum.loc[match_sum['team2'] == match_sum['winner']]
bat_chase.sample(2)

,team1,team2,winner,margin,matchDate,match_id,year,season
196,Sunrisers,Super Giants,Super Giants,5 wickets,2023-04-07,T209590,2023,3
157,RCB,Capitals,Capitals,7 wickets,2023-05-06,T208517,2023,3


In [77]:
bat_chase_win =  bat_chase.groupby(by=['winner'])[['winner']].count()
bat_chase_win.columns = ['win_chasing']
bat_chase_win = bat_chase_win.reset_index()
bat_chase_win = bat_chase_win.rename(columns = {'winner':'team'})
bat_chase_win = bat_chase_win.sort_values(by =['win_chasing'],ascending=False )
bat_chase_win.head(3)

,team,win_chasing
0,Capitals,14
1,KKR,14
9,Titans,14
